In [ ]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd

# 定義一個函數來獲取股票資料
def get_stock_data(stock_code):
    url = f'https://tw.stock.yahoo.com/quote/{stock_code}.TW'
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        # 查找包含特定文字的元素
        h1_tag = soup.find('h1', class_='C($c-link-text) Fw(b) Fz(24px) Mend(8px)')
        span_tag = soup.find('span', class_='C($c-icon) Fz(24px) Mend(20px)')

        # 股票名稱
        stock_name = h1_tag.text if h1_tag else 'N/A'

        # 股票代號
        stock_code_text = span_tag.text if span_tag else stock_code

        # 收盤日期
        closing_info = soup.find('span', class_='C(#6e7780) Fz(12px) Fw(b)')
        closing_date = 'N/A'
        if closing_info:
            closing_text = closing_info.text.strip()
            date_match = re.search(r'\d{4}/\d{2}/\d{2}', closing_text)
            if date_match:
                closing_date = date_match.group()

        # 股價、變動值和變動百分比
        price_div = soup.find('div', class_='D(f) Ai(fe) Mb(4px)')
        stock_price = change_value = percent_value = 'N/A'
        trend = 'N/A'

        if price_div:
            # 順序查找上漲、下跌和平盤的值
            price_value = price_div.find('span', class_='Fz(32px) Fw(b) Lh(1) Mend(16px) D(f) Ai(c) C($c-trend-up)')
            if not price_value:
                price_value = price_div.find('span', class_='Fz(32px) Fw(b) Lh(1) Mend(16px) D(f) Ai(c) C($c-trend-down)')
            if not price_value:
                price_value = price_div.find('span', class_='Fz(32px) Fw(b) Lh(1) Mend(16px) D(f) Ai(c)')

            change_value = price_div.find('span', class_='Fz(20px) Fw(b) Lh(1.2) Mend(4px) D(f) Ai(c) C($c-trend-up)')
            if not change_value:
                change_value = price_div.find('span', class_='Fz(20px) Fw(b) Lh(1.2) Mend(4px) D(f) Ai(c) C($c-trend-down)')
            if not change_value:
                change_value = price_div.find('span', class_='Fz(20px) Fw(b) Lh(1.2) Mend(4px) D(f) Ai(c)')

            percent_value = price_div.find('span', class_='Jc(fe) Fz(20px) Lh(1.2) Fw(b) D(f) Ai(c) C($c-trend-up)')
            if not percent_value:
                percent_value = price_div.find('span', class_='Jc(fe) Fz(20px) Lh(1.2) Fw(b) D(f) Ai(c) C($c-trend-down)')
            if not percent_value:
                percent_value = price_div.find('span', class_='Jc(fe) Fz(20px) Lh(1.2) Fw(b) D(f) Ai(c)')

            if price_value:
                if 'C($c-trend-down)' in price_value.get('class', []):
                    trend = '下跌'
                elif 'C($c-trend-up)' in price_value.get('class', []):
                    trend = '上漲'
                else:
                    trend = '平盤'
                stock_price = f"{float(price_value.text):.2f}"

            change_value = change_value.text.strip() if change_value else 'N/A'
            percent_value = re.sub(r'[\(\)%]', '', percent_value.text.strip()) if percent_value else 'N/A'

        return {
            '名稱': stock_name,
            '代號': stock_code_text,
            '股價': stock_price,
            '變動值': change_value,
            '變動百分比': percent_value,
            '趨勢': trend,
            '收盤日期': closing_date
        }
    else:
        print(f"無法檢索網頁，股票代碼 {stock_code}。狀態碼: {response.status_code}")
        return {
            '名稱': 'N/A',
            '代號': stock_code,
            '股價': 'N/A',
            '變動值': 'N/A',
            '變動百分比': 'N/A',
            '趨勢': 'N/A',
            '收盤日期': 'N/A'
        }

# 股票代號列表
stock_codes = ['2002', '2006', '2013', '2027', '2030', '2034', '2038', '2002A','2409','2451','2008']

# 獲取所有股票資料並存入 DataFrame
stock_data_list = []
for code in stock_codes:
    stock_data = get_stock_data(code)
    stock_data_list.append(stock_data)

df = pd.DataFrame(stock_data_list)
df

,名稱,代號,股價,變動值,變動百分比,趨勢,收盤日期
0,中鋼,2002,23.60,0.20,0.84,下跌,2024/06/04
1,東和鋼鐵,2006,69.90,0.00,0.14,平盤,2024/06/04
2,中鋼構,2013,58.10,0.00,0.17,平盤,2024/06/04
3,大成鋼,2027,37.80,0.36,0.92,下跌,2024/06/04
4,彰源,2030,18.45,0.05,0.27,下跌,2024/06/04
5,允強,2034,25.60,0.20,0.79,上漲,2024/06/04
6,海光,2038,20.45,0.35,1.68,下跌,2024/06/04
7,中鋼特,2002A,43.50,0.04,0.12,上漲,2024/06/04
8,友達,2409,17.50,0.30,1.69,下跌,2024/06/04
9,創見,2451,113.50,0.50,0.44,上漲,2024/06/04
